In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")

forward (generic function with 4 methods)

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/nominal_traj_mani_0708" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"]
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
ix = 3
iu = 2

2

In [5]:
dynamics = ThreeDOFManipulatorDynamics(9.81)

ThreeDOFManipulatorDynamics(6, 3, 3, 3, 3, 3, [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0 0 0; 0 0 0; … ; 0 1 0; 0 0 1], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.0, 0.0, 0.0], 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 9.81)

In [6]:
function diff_numeric_central(model::Dynamics,x::Vector,u::Vector)
    ix = length(x)
    iu = length(u)
    eps_x = Diagonal{Float64}(I, ix)
    eps_u = Diagonal{Float64}(I, iu)
    fx = zeros(ix,ix)
    fu = zeros(ix,iu)
    h = 2^(-18)
    for i in 1:ix
        fx[:,i] = (forward(model,x+h*eps_x[:,i],u) - forward(model,x-h*eps_x[:,i],u)) / (2*h)
    end
    for i in 1:iu
        fu[:,i] = (forward(model,x,u+h*eps_u[:,i]) - forward(model,x,u-h*eps_u[:,i])) / (2*h)
    end
    return fx,fu
end

diff_numeric_central (generic function with 1 method)

In [7]:
idx = 4
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])
A_,B_ = diff_numeric_central(dynamics,xnom[:,idx],unom[:,idx])

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0; … ; 12.474429758731276 9.545594508759677 … -4.136306210886687 -3.5286398231983185; -0.030982352793216705 11.05107393860817 … 6.7340164091438055 2.9852327425032854], [0.0 0.0 0.0; 0.0 0.0 0.0; … ; -0.4374258127063513 1.1089709997177124 0.4157871715724468; -1.1470838002860546 0.4157871715724468 4.970869882032275])

In [8]:
sum(abs.(A - A_))

1.3456373348930128e-8

In [9]:
sum(abs.(B - B_))

3.9234087090278535e-9

In [ ]:
sum(abs.(Sb - Fb))

In [10]:
sum(abs.(A))

176.54024501142624

In [11]:
sum(abs.(B))

10.78545700498374